In [ ]:
from bastionlab import Identity

# Create `Identity` for Data owner.
data_owner = Identity.create("data_owner")

In [ ]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

In [ ]:
!python datagen.py

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip

In [ ]:
!pip install transformers==0.13.2

In [5]:
import polars as pl

file_path = "./SMSSpamCollection"

labels = []
texts = []
with open(file_path) as f:
    for line in f.readlines():
        split = line.split("\t")
        labels.append(1 if split[0] == "spam" else 0)
        texts.append(split[1])
df = pl.DataFrame({"label": labels, "text": texts})
from bastionlab import Connection

connection = Connection("localhost")

from bastionlab.polars.policy import Policy, Aggregation, Log

policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())
rdf = connection.client.polars.send_df(
    df.limit(500), policy=policy, sanitized_columns=["text"]
)
test_rdf = connection.client.polars.send_df(
    df.tail(int(df.height / 3)).limit(250), policy=policy, sanitized_columns=["text"]
)

train_rds = rdf.convert(["text"], "distilbert-base-uncased").collect()
rdf = train_rds.to_dataset(["text_ids", "text_mask"], "label")

rdf
# test_rds = test_rdf.to_dataset(["text"], "label", "distilbert-base-uncased")

# train_rds._set_test_dataset(test_rds.train_dataset_ref)
# test_rds.set_train_dataset(train_dataset=train_rds.train_dataset_ref)

In [ ]:
! pip install transformers

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from bastionlab.torch.utils import MultipleOutputWrapper

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(model, 0)

In [ ]:
remote_datasets = connection.client.torch.list_remote_datasets()
print(remote_datasets)

print([r.trace_input for r in remote_datasets])

In [ ]:
from bastionlab.torch.optimizer_config import Adam
remote_learner = connection.client.torch.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
)

In [ ]:
import polars as pl

df = pl.DataFrame(
    {
        "0": [1, 2, 3, 4, 5, 6]
    }
)

df.get_column("0").reshape([2,3])